# Introduction To NER

![대체 텍스트](https://i.imgur.com/kcucuJ6.png)

개체명 인식(Named Entity Recognition, NER)이란 '이름을 가진 개체(named entity)'를 찾아내고 이를 인명, 기관명, 지명 등의 유형으로 인식하는 것을 말합니다. 

각 개체명은 문장 또는 문서에서 특정한 의미를 가지기 때문에 정보 검색 및 언어 이해를 위한 분석에서 주요한 대상으로 다뤄집니다. 

본 글은 NER를 이해하기 위한 시리즈의 첫 번째 글로 앞으로의 글 목록은 다음과 같습니다:

1. Introduction To NER
2. NER With CRF In Python
3. NER with LSTM
4. Sequence Tagging With A LSTM-CRF
... 

데이터셋은 [kaggle](https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus)에서 가져왔습니다. 



In [0]:
import pandas as pd
import numpy as np

data = pd.read_csv("/content/ner_dataset.csv", encoding='latin1')

In [0]:
# 빈 값을 앞 데이터로부터 채우기
# https://ordo.tistory.com/59

data = data.fillna(method="ffill")

In [0]:
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [0]:
words = list(set(data["Word"].values))
n_words = len(words)

print(n_words)

35178


data.tail()과 n_words를 통해 알아낸 데이터의 특징:
- 문장의 개수: 46,959개
- 단어 개수 : 35,178개

그렇다면 이제 데이터를 [문장] : [POS] : [NER]의 형태로 만들어봅시다. 



In [0]:
def GetSentencePair(data):
    n_sent = 1
    s = data[data["Sentence #"] == "Sentence: {}".format(n_sent)]
    n_sent +=1
    return s["Word"].values.tolist(), s["POS"].values.tolist(), s["Tag"].values.tolist() 

In [0]:
sent, pos, tag = GetSentencePair(data)

In [0]:
print(sent)
print(pos)
print(tag)

['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']
['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP', 'TO', 'VB', 'DT', 'NN', 'IN', 'NNP', 'CC', 'VB', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'DT', 'NN', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


## A simple machine learning approach

첫 단계로 단어를 모두 벡터로 바꾸어주어야 합니다. 
그 후, 사이킷런(sklearn)의 랜덤 포레스트 분류기(RamdomForestClassifier)를 이용해 단어를 분류합니다.

랜덤 포레스트를 이용하기 위해 각 단어의 피쳐들을 선정해 값을 넣어줄 것입니다. 

고려하는 피쳐의 목록은 다음과 같습니다:
- word.istitle() : 대문자로 시작
- word.islower() : 모든 문자 소문자
- word.isupper() : 모든 문자 대문자
- len(word) : 단어 길이
- word.isdigit()  : 숫자
- word.isalpha() : 알파벳



In [0]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier

In [0]:
def feature_map(word):
    '''Simple feature map.'''
    return np.array([word.istitle(), word.islower(), word.isupper(), len(word),
                     word.isdigit(),  word.isalpha()])

In [0]:
words = [feature_map(word) for word in data["Word"].values.tolist()]
tags = data["Tag"].values.tolist()

In [0]:
print(words[0])

[1 0 0 9 0 1]


In [0]:
# X: The data to fit 
# y: The target variable to try to predict
# cv : Determines the cross-validation splitting strategy.  5-fold cross validation

pred = cross_val_predict(RandomForestClassifier(n_estimators=20), X= words, y=tags, cv=5)
                                            

In [0]:
report = classification_report(y_pred=pred, y_true=tags)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
print(report)

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00       402
       B-eve       0.00      0.00      0.00       308
       B-geo       0.26      0.79      0.40     37644
       B-gpe       0.25      0.06      0.09     15870
       B-nat       0.00      0.00      0.00       201
       B-org       0.65      0.17      0.27     20143
       B-per       0.96      0.20      0.33     16990
       B-tim       0.29      0.32      0.30     20333
       I-art       0.00      0.00      0.00       297
       I-eve       0.00      0.00      0.00       253
       I-geo       0.00      0.00      0.00      7414
       I-gpe       0.00      0.00      0.00       198
       I-nat       0.00      0.00      0.00        51
       I-org       0.36      0.03      0.06     16784
       I-per       0.47      0.02      0.04     17251
       I-tim       0.50      0.06      0.11      6528
           O       0.97      0.98      0.97    887908

    accuracy              

결과가 매우 나쁘죠? 이는 결정에 필요한 피쳐들이 충분하지 않았기 때문입니다. 따라서, 다음 포스트에서는 보다 정교한 알고리즘을 이용해 피쳐를 선정해봅시다!